# 1. Μέρος 1ο – Ταξινόμηση με Συνελικτικά Νευρωνικά Δίκτυα (60%)
### d. Πειραματιστείτε και προτείνετε ένα CNN δικής σας έμπνευσης που να βελτιστοποιεί την ακρίβεια ταξινόμησης στο test set χωρίς ενδείξεις overfitting.

Για λόγους σύγκρισης πραγματοποιήθηκε και pre trained CNN που δείχνει πόσο γρήγορα και καλά αποδίδουν, καθώς έχουν ήδη γίνει trained στην ImageNet.

In [ ]:
import numpy as np 
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD,Adam,lr_scheduler
from torch.utils.data import random_split
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [ ]:
train_transform = transforms.Compose([transforms.Resize(size=(224, 224)),
                                      transforms.RandomHorizontalFlip(p = 0.4), # FLips the image w.r.t horizontal axis
                                      transforms.RandomRotation(30), #Rotates the image to a specified angel
                                      transforms.Pad(4),
                                      transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), #Performs actions like zooms, change shear angles.
                                      transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # Set the color params
                                      transforms.ToTensor(), # comvert the image to tensor so that it can work with torch
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) #Normalize all the images
                               ])
 
 
test_transform = transforms.Compose([transforms.Resize(size=(224, 224)),
                               transforms.Pad(4),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                               ])

In [ ]:
train_data = datasets.CIFAR10(root='data', train=True, download=True, transform=train_transform)
test_data = datasets.CIFAR10(root='data', train=False, download=True, transform=test_transform)

trainloader = DataLoader(dataset=train_data, 
                          batch_size=40,
                          num_workers=2,
                          shuffle=True)

testloader = DataLoader(dataset=test_data, 
                         batch_size=40,
                         num_workers=2,
                         shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
from torchvision import models

In [ ]:
model = models.vgg16(pretrained = True)
input_lastLayer = model.classifier[6].in_features
model.classifier[6] = nn.Linear(input_lastLayer,10)



/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
model


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum=0.9,weight_decay=5e-4)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
epochs = 4

In [ ]:

for epoch in range(epochs):
   for i, (imgs , labels) in enumerate(trainloader):
       imgs = imgs.to(device)
       labels = labels.to(device)

       labels_hat = model(imgs)
       n_corrects = (labels_hat.argmax(axis=1)==labels).sum().item()
       loss_value = criterion(labels_hat, labels)
       loss_value.backward()
       optimizer.step()
       optimizer.zero_grad()
       if (i+1) % 250 == 0:
           print(f'epoch {epoch+1}/{epochs}, step: {i+1}/{len(trainloader)}: loss = {loss_value:.5f}, acc = {100*(n_corrects/labels.size(0)):.2f}%')
   print()

epoch 1/4, step: 250/1250: loss = 0.83605, acc = 60.00%
epoch 1/4, step: 500/1250: loss = 0.65351, acc = 75.00%
epoch 1/4, step: 750/1250: loss = 0.49315, acc = 82.50%
epoch 1/4, step: 1000/1250: loss = 0.52730, acc = 85.00%
epoch 1/4, step: 1250/1250: loss = 0.30114, acc = 85.00%

epoch 2/4, step: 250/1250: loss = 0.57333, acc = 80.00%
epoch 2/4, step: 500/1250: loss = 0.36119, acc = 85.00%
epoch 2/4, step: 750/1250: loss = 0.48034, acc = 85.00%
epoch 2/4, step: 1000/1250: loss = 0.11929, acc = 97.50%
epoch 2/4, step: 1250/1250: loss = 0.24122, acc = 92.50%

epoch 3/4, step: 250/1250: loss = 0.29677, acc = 90.00%
epoch 3/4, step: 500/1250: loss = 0.47425, acc = 82.50%
epoch 3/4, step: 750/1250: loss = 0.45379, acc = 82.50%
epoch 3/4, step: 1000/1250: loss = 0.31214, acc = 87.50%
epoch 3/4, step: 1250/1250: loss = 0.36431, acc = 85.00%

epoch 4/4, step: 250/1250: loss = 0.40915, acc = 85.00%
epoch 4/4, step: 500/1250: loss = 0.33769, acc = 85.00%
epoch 4/4, step: 750/1250: loss = 0.262

Παρατηρούμε, όπως είναι αναμενόμενο, πως το pre trained δίκτυο αποδίδει πολύ καλύτερα σε πολύ λιγότερο χρόνο. 